In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46391
2,Huelva,Confirmados PDIA 14 días,400
3,Huelva,Tasa PDIA 14 días,"77,94687920182396"
4,Huelva,Confirmados PDIA 7 días,210
5,Huelva,Tasa PDIA 7 dias,"40,922111580957576"
6,Huelva,Total Confirmados,46594
7,Huelva,Curados,45267
8,Huelva,Fallecidos,420


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46391.0


/tmp/ipykernel_85372/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12745.0


/tmp/ipykernel_85372/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_85372/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_85372/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_85372/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2664 personas en los últimos 7 días 

Un positivo PCR cada 1483 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46391.0,210.0,400.0,513170.0,40.922112,77.946879,89.0
Huelva-Costa,27426.0,107.0,222.0,289548.0,36.954149,76.671225,44.0
Condado-Campiña,14429.0,85.0,145.0,156231.0,54.406616,92.811286,36.0
Huelva (capital),12745.0,54.0,97.0,143837.0,37.542496,67.437447,25.0
Bollullos Par del Condado,1311.0,13.0,15.0,14387.0,90.359352,104.260791,13.0
Almonte,2343.0,17.0,26.0,24507.0,69.367936,106.092137,11.0
Sierra de Huelva-Andévalo Central,4110.0,18.0,33.0,67391.0,26.709798,48.967963,10.0
Cartaya,2153.0,28.0,63.0,20083.0,139.421401,313.698153,9.0
Isla Cristina,3175.0,14.0,22.0,21393.0,65.441967,102.837377,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,58.0,6.0,13.0,713.0,841.514727,1823.281907,5.0
Cartaya,2153.0,28.0,63.0,20083.0,139.421401,313.698153,9.0
Almonaster la Real,90.0,4.0,5.0,1788.0,223.713647,279.642058,3.0
San Bartolomé de la Torre,388.0,3.0,9.0,3761.0,79.766020,239.298059,0.0
Palos de la Frontera,1088.0,17.0,28.0,11742.0,144.779424,238.460228,7.0
Minas de Riotinto,189.0,3.0,8.0,3812.0,78.698846,209.863589,1.0
Moguer,2041.0,18.0,34.0,21867.0,82.315818,155.485435,1.0
Almonte,2343.0,17.0,26.0,24507.0,69.367936,106.092137,11.0
Bollullos Par del Condado,1311.0,13.0,15.0,14387.0,90.359352,104.260791,13.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,2112.0,2.0,13.0,21104.0,9.476876,61.599697,0.0,0.153846
San Bartolomé de la Torre,388.0,3.0,9.0,3761.0,79.766020,239.298059,0.0,0.333333
Aljaraque,1647.0,5.0,14.0,21474.0,23.283971,65.195120,1.0,0.357143
Minas de Riotinto,189.0,3.0,8.0,3812.0,78.698846,209.863589,1.0,0.375000
Trigueros,545.0,2.0,5.0,7862.0,25.438820,63.597049,0.0,0.400000
Lepe,3050.0,3.0,7.0,27880.0,10.760402,25.107604,2.0,0.428571
Cartaya,2153.0,28.0,63.0,20083.0,139.421401,313.698153,9.0,0.444444
Campofrío,58.0,6.0,13.0,713.0,841.514727,1823.281907,5.0,0.461538
Huelva-Costa,27426.0,107.0,222.0,289548.0,36.954149,76.671225,44.0,0.481982
